<a href="https://colab.research.google.com/github/xixihaha1995/esp_proj3/blob/main/_2_gradient_descent_and_backpropogation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# Introduction to Backpropogation and Gradient descent

## Cost function
$y_{pred} = a + bx + cx^2 + dx^3$ </br>
$Los\quad or \quad Cost = (y_{pred} - y_{true})^2$</br>
How should we change a, b, c, d to minimize the prediction loss?

## Gradient descent
(5 minutes) Gradient descent: https://youtu.be/IHZwWFHWa-w?t=181 </br>

## Chain rules
[3Blue1brown](https://youtu.be/tIeHLnjs5U8?t=234)
![](https://drive.google.com/uc?export=view&id=1xAOIp0B0qwppS_zS4BItax4RjdDwQtfL)

## How to change a, b, c, d
$y_{pred} = a + bx + cx^2 + dx^3$ </br>
$\frac{\partial loss}{\partial y_{pred}} = \frac{\partial (y_{pred} - y)^2}{\partial y_{pred}} = 2(y_{pred} - y)$</br>
$\frac{\partial loss}{\partial d} = \frac{\partial loss}{\partial y_{pred}} \frac{\partial y_{pred}}{\partial d}= \frac{\partial loss}{\partial y_{pred}}x^3 =2(y_{pred} - y)x^3$</br>


## (Optional)Stochastic Graduebt descebt, Backpropagation
Tools: https://www.derivative-calculator.net/ </br>

(3 minutes) [**Batches**] Stochastic Gradient descent: https://youtu.be/Ilg3gGewQ5U?t=573 </br>
(Optional) A [Visual Explanation](https://towardsdatascience.com/a-visual-explanation-of-gradient-descent-methods-momentum-adagrad-rmsprop-adam-f898b102325c) of Gradient Descent Methods.</br> 
(Optional) Why Momentum Really Works: https://distill.pub/2017/momentum/


Training based on backpropogation
----------------

A third order polynomial, trained to predict $y=\sin(x)$ from $-\pi$
to $pi$ by minimizing squared Euclidean distance.

This implementation uses PyTorch tensors to manually compute the forward pass,
loss, and backward pass.

A PyTorch Tensor is basically the same as a numpy array: it does not know
anything about deep learning or computational graphs or gradients, and is just
a generic n-dimensional array to be used for arbitrary numeric computation.

The biggest difference between a numpy array and a PyTorch Tensor is that
a PyTorch Tensor can run on either CPU or GPU. To run operations on the GPU,
just cast the Tensor to a cuda datatype. </br>
More [references](https://pytorch.org/tutorials/index.html).


In [ ]:
import torch
import math

'''
Hyperparameters
'''
learning_rate = 1e-6
EPOCHS = 2000

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)


for t in range(EPOCHS):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 500 == 99:
        print(f'Epoch #{t}, loss:{loss:.2f}')

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred -y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

Epoch #99, loss:2158.47
Epoch #599, loss:284.23
Epoch #1099, loss:44.28
Epoch #1599, loss:13.42
Result: y = -0.010597323067486286 + 0.8291981220245361 x + 0.0018282142700627446 x^2 + -0.08941266685724258 x^3


## Prediction perfomance

### CVRMSE
According to American Society of Heating, Refrigerating and Air-Conditioning Engineers (ASHRAE) Guideline I4-2014, Measurement of Energy, Demand, and Water Savings:<br>
CV-(RMSE): the coefficient of variation (CV) of the root-mean-square error (RMSE); an indication of how much variation or randomness there is between the data and the model, calculated by the average energy use.<br>
The RMSE of predicted values $\hat y$ for times t of a regression's dependent variable $y_t$, with variables observed over $T$ times, is computed for $T$ different predictions as the square root of the mean of the squares of the deviations(errors):<br>
$RMSE = \sqrt{\frac{\sum_{t=1}^{T}(\hat y_t - y_t)^2}{T}}$<br>
$CVRMSE = \frac{RMSE}{\bar y}$<br>
where $\bar y$ is the mean value of measurements (or the average energy use in the building performance field)

### MAPE
The mean absolute percentage error (MAPE), also known as mean absolute percentage deviation (MAPD), is a measure of prediction accuracy of a **forecasting** method in statistics. It usually expresses the accuracy as a ratio defined by the formula:<br>
$MAPE = \frac{100\%}{T}\sum_{t=1}^T|\frac{y_t-\hat y_t}{y_t}|$

In [ ]:
import numpy as np
def cvrmse_cal(measure, predict, mean_measured):
    rmse = (sum((measure - predict) ** 2) / len(measure)) ** (1 / 2)
    cvrmse = rmse * 100 / mean_measured
    return cvrmse


def mean_absolute_percentage_error(measure, model):
  # print(abs(measure - model))
  # print(abs(measure))
  nom = abs(measure - model) / abs(measure)
  # nom[nom == float('inf')] = 0
  # new_nom = []
  # for num in nom:
  #     if (np.isnan(num)):
  #         new_nom.append(0)
  #     else:
  #         new_nom.append(num)
  # print(sum(new_nom))
  return sum(nom)*100 / len(measure)

In [ ]:
y_pred = a + b * x + c * x ** 2 + d * x ** 3
cvrmse = cvrmse_cal(abs(y), abs(y_pred), abs(y).mean())
mape = mean_absolute_percentage_error(y, y_pred)
import pandas as pd
dataset = pd.DataFrame({'y_pred': y_pred.numpy(), 
                        'y': y.numpy()})

In [ ]:
import plotly.express as px

fig = px.line(dataset, y=['y_pred','y'])
fig.update_layout(
     title=f"Prediction Performance: <br>"
     f"Loss:<b>{loss:.2f}</b>;"
     f"CVRMSE:<b>{cvrmse:.2f}</b>%;"
     f"MAPE:<b>{mape:.2f}</b>%",
     yaxis_title="y",
     xaxis_title="x",
     legend_title="Data types",
     font=dict(
         family="Times New Roman",
         size=20,
         color="Black"
         )
     )
fig.show()